# Fit a line to the middle part of the finger 1 motor positions

It might be helpful to look at the instruction/background slides before tackling this

https://docs.google.com/presentation/d/1IoiD43zjp_XOEH0soCZGMBN4b13KWVndoHAxoUJMQoM/edit?usp=sharing

Lecture goals:
- Extract a subset of data based on conditions
- Use linregress to approximate linear data

In [ ]:
# Doing the imports for you
import numpy as np
import json as json
from scipy.stats import linregress
import matplotlib.pyplot as plt

# Read the motor position data in
The motor position data has already been cut out and put in separate .csv files for you (see Lecture script, slides). 

TODO: Read the data in using numpy's loadtxt.

The shape of the data should be (355, 40) and (305, 40)

In [ ]:
# BEGIN SOLUTION NO PROMPT
# Read in the processed data

data_successful = np.loadtxt("Data/week3_Motor position f1_successful.csv", dtype="float", delimiter=",")
data_failed = np.loadtxt("Data/week3_Motor position f1_failed.csv", dtype="float", delimiter=",")
print(f"{data_successful.shape}")
print(f"{data_failed.shape}")

# END SOLUTION

""" # BEGIN PROMPT
# TODO Read in week3_Motor position f1_success and failed .csv files
data_successful = ...  # week3_Motor position f1_successful.csv
data_failed = ...  # week3_Motor position f1_failed.csv
# END PROMPT"""


In [ ]:
assert(data_successful.shape == (355, 40) and data_failed.shape == (305, 40))

# Fit a line to the middle points

Doing this as a function so you can use it twice.

Some decisions:
- Pass in the data as two arrays, the t and y values
-     Use fit_line_to_middle_bit function to pull out the "middle" bit
- Returns the points of intersection with the min/max values (rather than slope and intercept) because in the long run that's what we care about (where the motor started and stopped)
- The eps is a "fudge factor" so that you can clip out data above/below a threshhold

Two ways to do this:
- Clip with a fudge factor, use np.logical_and fudge factor should be d_y = eps * (y_max - y_min), 
   take all points y_min + d_y < y < y_max - dy
- Use np.where to find the first index > y_min + d_y (or < y_max - d_y)
    np.where returns an array of arrays; use index[0][0] to get the value out
    Then use start:end to get the values out of ts, ys

## Implementation steps (suggested)

Step 1: Start with just fitting the entire data and drawing the resulting line
    Use y = mx + b equation to find start/stop points from slope/intercept
       Find max/min y values, and plug those into the equation
    You should get something that goes roughly from the bottom left to the top right of the original data

Step 2: Clip just the y_max values by using boolean indexing to keep just part of the array
   Alternate: Use np.where to find the max value, extract the ending index, and use 0:end_index

Step 3: Clip both ends at the same time (change the boolean indexing to use a logical_and)
    Alternate: Use np.where a second time to find the min value, extract the starting index

In [ ]:
def fit_line_to_middle_bit(ts, ys, eps=1e-2):
    """ Fit a line to the sloped middle bit of the data
      Return the line as a pair of points, one where the fitted line crosses the y min value, one where the line crosses
      the y max value (see slides for picture)
      Reminder; Use & to do an AND of two boolean conditions
    @param ts - the time values for the data (x-axis)
    @param ys - the function values (y-axis)
    @param eps - a fudge factor for clipping the middle bit
    @returns (x_min, y_min), (x_max, y_max)  (two tuples)"""

    # BEGIN SOLUTION NO PROMPT

    y_max = np.max(ys)
    y_min = np.min(ys)
    delta_y = eps * (y_max - y_min)

    find_vals = np.logical_and(ys > (y_min + delta_y), ys < (y_max - delta_y))
    res_clip = linregress(ts[find_vals], ys[find_vals])

    index_start = np.where(ys > y_min + delta_y)
    index_end = np.where(ys > y_max - delta_y)
    res_bounds = linregress(ts[index_start[0][0]:index_end[0][0]], ys[index_start[0][0]:index_end[0][0]])

    # print(f"Result, clipping: {res_clip}")
    # print(f"Result, bounds: {res_bounds}")

    x_min = (y_min - res_clip.intercept) /res_clip.slope
    x_max = (y_max - res_clip.intercept) /res_clip.slope
    # END SOLUTION

    """ # BEGIN PROMPT
    # TODO: 
    #.  Get y min and y max from the data
    #.  Use np.where to find the index of the first y value that is bigger than ymin + eps * (ymax - ymin)
    #.  Use np.where to find the index of the last y value that is smaller than ymax - eps * (ymax - ymin)
    #  (Check - if you've done this correctly, for the first row of the successful data set this should be around
    #.   8 and 12
    #   Get out just the data between those two indices
    #.  Do a linear regression on just that data
    #.     Don't forget to get the t values between those two indices as well
    #. After fitting, Slope should be 654.xxx, intercept 162.xxx for first row (successful data
    #. Use the fitted line to get out the corresponding x values for those y values
    #.   x = (y - intercept) / slope
    y_min = ...
    y_max = ...
    x_min = ...
    x_max = ...
    # END PROMPT"""
    return (x_min, y_min), (x_max, y_max)

# BEGIN SOLUTION NO PROMPT
# This bit of code will test your function with known values - should print out ((0.25, 0.0), (0.75, 1.0))
xs = np.linspace(0, 1, 9)
ys = np.array([0, 0, 0, 0.25, 0.5, 0.75, 1, 1, 1])
print(f"{fit_line_to_middle_bit(xs, ys, eps=0.1)}")
# END SOLUTION

In [ ]:
assert(fit_line_to_middle_bit(np.linspace(0, 1, 9), np.array([0, 0, 0, 0.25, 0.5, 0.75, 1, 1, 1]), eps=0.001) == ((0.25, 0.0), (0.75, 1.0)))

# Use the function on each data set
Call the function twice, once with the successful pick data, once with the unsuccessful.

TODO
- Create t values (time step is 1/30th of a second, make one t value for each y value, starting at 0
- Call the **fit_line_to_middle_bits** function twice, once for the first row of the successful data, once for the first row of the failed data

In [ ]:
# BEGIN_SOLUTION NO PROMPT
# Know time step
time_step = 1/30
ts = np.arange(start=0, stop=data_successful.shape[1] * time_step, step=time_step)

pt_start_successful, pt_end_successful = fit_line_to_middle_bit(ts, data_successful[0])
pt_start_failed, pt_end_failed = fit_line_to_middle_bit(ts, data_failed[0])
# END SOLUTION NO PROMPT

""" # BEGIN PROMPT
# Create t values with appropriate step size
ts = ...
pt_start_successful, pt_end_successful = ...
pt_start_failed, pt_end_failed = ...
# END PROMPT"""

print(f"Last time step {ts[-1]}")
print(f"Successful: {pt_start_successful} {pt_end_successful}")
print(f"Failed: {pt_start_failed} {pt_end_failed}")

In [ ]:
assert(np.isclose(ts[-1], 1.3))

In [ ]:
assert(np.isclose(pt_start_successful[0], 0.210644, atol=1e-2))

In [ ]:
assert(np.isclose(pt_start_successful[1], 299.992, atol=1e-1))

In [ ]:
assert(np.isclose(pt_end_successful[0], 0.409512247, atol=1e-2))

In [ ]:
assert(np.isclose(pt_end_successful[1], 429.88, atol=1e-1))

In [ ]:
assert(np.isclose(pt_start_failed[1], 299.992, atol=1e-1))

In [ ]:
assert(np.isclose(pt_end_failed[1], 429.79199, atol=1e-1))

# Plot the results

Plot the successful data on the left, the failed on the right.

For each plot, plot the data, the fitted line, and red X's for the start/stop end points of the line (see slides)

In [ ]:
n_rows = 1
n_cols = 2
fig, axs = plt.subplots(n_rows, n_cols)

# BEGIN_SOLUTION NO_PROMPT
# Know time step
time_step = 1/30
ts = np.arange(start=0, stop=data_successful.shape[1] * time_step, step=time_step)

pt_start, pt_end = fit_line_to_middle_bit(ts, data_successful[0])
axs[0].plot(ts, data_successful[0], '-b', label=f"Motor position f1")
axs[0].plot([pt_start[0], pt_end[0]], [pt_start[1], pt_end[1]], ':k', label="fitted line")
axs[0].plot([pt_start[0], pt_end[0]], [pt_start[1], pt_end[1]], 'Xr')
axs[0].set_title(f"Succesful t={pt_end[0]}")
axs[0].legend()

pt_start, pt_end = fit_line_to_middle_bit(ts, data_failed[0])
axs[1].plot(ts, data_successful[0], '-b', label=f"Motor position f1")
axs[1].plot([pt_start[0], pt_end[0]], [pt_start[1], pt_end[1]], ':k', label="fitted line")
axs[1].plot([pt_start[0], pt_end[0]], [pt_start[1], pt_end[1]], 'Xr')
axs[1].set_title(f"Failed t={pt_end[0]}")
axs[1].legend()

# END_SOLUTION

""" # BEGIN PROMPT
# Create t values with appropriate step size
ts = ...
# TODO Plot original data and fitted line (see slides)
#  TODO: Put the end time value in the title
# END PROMPT"""
plt.tight_layout()


In [ ]:
assert(len(axs[0].lines) == 3 and len(axs[1].lines) == 3)

In [ ]:
assert("0.40" in axs[0].get_title())

In [ ]:
assert("0.41" in axs[1].get_title())

## Hours and collaborators
Required for every assignment - fill out before you hand-in.

Listing names and websites helps you to document who you worked with and what internet help you received in the case of any plagiarism issues. You should list names of anyone (in class or not) who has substantially helped you with an assignment - or anyone you have *helped*. You do not need to list TAs.

Listing hours helps us track if the assignments are too long.

In [ ]:
# BEGIN SOLUTION NO PROMPT
worked_with_names = {}
websites = {}
hours = 4

# END SOLUTION

""" # BEGIN PROMPT
# List of names (creates a set)
worked_with_names = {"not filled out"}
# List of URLS (creates a set)
websites = {"not filled out"}
# Approximate number of hours, including lab/in-class time
hours = -1.5

# for all row, column in all_indices_from_where
#.   if this is the column for wrist torque 
#.      print(f"Row: {r}, Time step: {c // n_time_steps} Successful y/n: {pick_data[r, -1] == 1}, value: {pick_data[r, c]}")
""" # END PROMPT

In [ ]:
assert(not "not filled out" in worked_with_names)

In [ ]:
assert(not "not filled out" in websites)

In [ ]:
assert(hours > 0)